# 第16章：Cross-Thread Memory（Store）

## 学习目标

本章将学习：
1. Store接口和长期记忆概念
2. InMemoryStore的使用
3. namespace（命名空间）的组织方式
4. put()、get()、search()方法
5. Store与Checkpointer的区别
6. 在Agent工具中使用Store
7. 语义搜索功能
8. 实战：记住用户偏好的智能助手

---

## 为什么需要Store？

### Checkpointer的局限

在第15章中，我们学习了Checkpointer实现**短期记忆（Thread-level Persistence）**：

```python
# Thread 1: 用户A的对话
agent.invoke(..., {"configurable": {"thread_id": "user-a-thread-1"}})

# Thread 2: 用户A的另一次对话
agent.invoke(..., {"configurable": {"thread_id": "user-a-thread-2"}})
```

**问题**：
- ❌ 不同Thread之间**无法共享信息**
- ❌ 用户在Thread 1说"我喜欢Python"，Thread 2中Agent不记得
- ❌ 无法存储跨会话的用户档案、偏好等

### Store解决方案

**Store实现跨Thread的长期记忆**：

```python
# Thread 1: 用户A告诉Agent偏好
store.put(("user-a",), "preferences", {"language": "Python"})

# Thread 2: 用户A的新对话，Agent可以读取偏好
prefs = store.get(("user-a",), "preferences")
# prefs.value = {"language": "Python"}
```

**优势**：
- ✅ 跨Thread共享信息
- ✅ 持久化用户偏好、档案
- ✅ 支持语义搜索
- ✅ 组织结构化记忆（namespace）

---

## 核心概念对比

| 特性 | Checkpointer（短期记忆） | Store（长期记忆） |
|------|-------------------------|------------------|
| **作用域** | Thread内 | 跨Thread |
| **内容** | 对话历史、AgentState | 用户偏好、档案、知识 |
| **生命周期** | Thread存在期间 | 永久（或手动删除） |
| **访问方式** | 自动加载/保存 | 显式调用put/get |
| **典型用途** | 多轮对话 | 用户画像、个性化 |
| **实现** | InMemorySaver | InMemoryStore |

**关系**：
- Checkpointer + Store = 完整的Agent记忆系统
- Checkpointer管理"短期工作记忆"
- Store管理"长期知识记忆"

---

## Store的数据模型

### 1. 存储结构

Store将数据存储为**JSON文档**，按以下方式组织：

```
Store
├─ Namespace: ("user-alice",)
│  ├─ Key: "preferences"    → Value: {"lang": "Python"}
│  ├─ Key: "profile"        → Value: {"name": "Alice"}
│  └─ Key: "history"        → Value: {...}
│
├─ Namespace: ("user-bob",)
│  ├─ Key: "preferences"    → Value: {"lang": "JavaScript"}
│  └─ Key: "profile"        → Value: {"name": "Bob"}
│
└─ Namespace: ("user-alice", "memories")
   ├─ Key: "memory-1"       → Value: {"text": "喜欢吃披萨"}
   └─ Key: "memory-2"       → Value: {"text": "是工程师"}
```

### 2. Namespace（命名空间）

- **定义**：类似"文件夹"，用于组织和隔离数据
- **格式**：Tuple，如`("user-id",)`或`("user-id", "context")`
- **作用**：
  - 按用户隔离：`("user-123",)`
  - 按类别组织：`("user-123", "preferences")`
  - 层级结构：`("org-1", "team-a", "user-123")`

### 3. Key（键）

- **定义**：Namespace内的唯一标识符，类似"文件名"
- **格式**：字符串，如`"preferences"`、`"profile"`

### 4. Value（值）

- **定义**：存储的实际数据
- **格式**：任意JSON可序列化的对象（dict, list, str等）

In [1]:
# 环境配置
import os
import sys

_project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(_project_root)

from config import config
from langchain_openai import ChatOpenAI
from langchain.tools import tool, ToolRuntime
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.memory import InMemoryStore
from dataclasses import dataclass

# 初始化模型
model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=config.CLOUD_API_KEY,
    base_url=config.CLOUD_BASE_URL,
)

print("环境配置完成！")
print(f"模型: {model.model_name}")

环境配置完成！
模型: gpt-4.1-mini


## 1. Store基础操作

### 创建Store和基本CRUD

In [2]:
print("【演示：Store基础操作】")
print()

# 创建Store（简化版，不带语义搜索）
store = InMemoryStore()

# 定义namespace
namespace_alice = ("user-alice",)
namespace_bob = ("user-bob",)

print("【1. PUT - 存储数据】")
# put(namespace, key, value)
store.put(namespace_alice, "preferences", {
    "language": "Python",
    "style": "简洁",
    "timezone": "Asia/Shanghai"
})
print("✓ 存储Alice的偏好")

store.put(namespace_bob, "preferences", {
    "language": "JavaScript",
    "style": "详细"
})
print("✓ 存储Bob的偏好")
print()

print("【2. GET - 读取数据】")
# get(namespace, key) 返回Item对象
item = store.get(namespace_alice, "preferences")
print(f"Alice的偏好: {item.value}")

item = store.get(namespace_bob, "preferences")
print(f"Bob的偏好: {item.value}")
print()

print("【3. 不存在的Key】")
item = store.get(namespace_alice, "non-existent")
print(f"不存在的Key: {item}")  # 返回None
print()

print("【4. 多条数据】")
store.put(namespace_alice, "profile", {
    "name": "Alice",
    "age": 28,
    "job": "Engineer"
})
print("✓ 存储Alice的档案")
print()

print("说明: Namespace隔离数据，每个用户有独立的存储空间")

【演示：Store基础操作】

【1. PUT - 存储数据】
✓ 存储Alice的偏好
✓ 存储Bob的偏好

【2. GET - 读取数据】
Alice的偏好: {'language': 'Python', 'style': '简洁', 'timezone': 'Asia/Shanghai'}
Bob的偏好: {'language': 'JavaScript', 'style': '详细'}

【3. 不存在的Key】
不存在的Key: None

【4. 多条数据】
✓ 存储Alice的档案

说明: Namespace隔离数据，每个用户有独立的存储空间


### Namespace的层级组织

In [3]:
print("【演示：Namespace的层级结构】")
print()

# 单层namespace
namespace_1 = ("user-alice",)

# 两层namespace：用户+上下文
namespace_2 = ("user-alice", "work")
namespace_3 = ("user-alice", "personal")

# 三层namespace：组织+团队+用户
namespace_4 = ("org-acme", "team-eng", "user-alice")

# 存储到不同namespace
store.put(namespace_2, "tasks", ["写代码", "开会", "Review PR"])
store.put(namespace_3, "tasks", ["买菜", "健身", "读书"])

print("【Alice的工作任务】")
work_tasks = store.get(namespace_2, "tasks")
print(f"  {work_tasks.value}")

print("\n【Alice的个人任务】")
personal_tasks = store.get(namespace_3, "tasks")
print(f"  {personal_tasks.value}")

print()
print("说明: Namespace支持层级结构，灵活组织数据")

【演示：Namespace的层级结构】

【Alice的工作任务】
  ['写代码', '开会', 'Review PR']

【Alice的个人任务】
  ['买菜', '健身', '读书']

说明: Namespace支持层级结构，灵活组织数据


## 2. 在Agent中使用Store

### Agent如何访问Store？

通过**工具**访问Store，使用`ToolRuntime`参数：

```python
@tool
def save_preference(key: str, value: str, runtime: ToolRuntime) -> str:
    """保存用户偏好"""
    user_id = runtime.context.user_id  # 从runtime获取用户ID
    store = runtime.store              # 从runtime获取Store
    
    store.put((user_id,), key, value)  # 存储数据
    return f"已保存偏好: {key} = {value}"
```

### Context Schema

需要定义`Context`来传递用户ID等信息：

```python
@dataclass
class Context:
    user_id: str

agent = create_agent(
    model=model,
    tools=[save_preference],
    context_schema=Context,  # 指定Context类型
    store=store              # 传入Store
)

agent.invoke(..., context=Context(user_id="user-123"))
```

In [8]:
print("【演示：Agent中使用Store】")
print()

# 定义Context
@dataclass
class UserContext:
    user_id: str

# 创建Store
user_store = InMemoryStore()

# 定义存储偏好的工具
@tool
def save_user_preference(preference_key: str, preference_value: str, runtime: ToolRuntime[UserContext]) -> str:
    """保存用户偏好到长期记忆"""
    user_id = runtime.context.user_id
    store = runtime.store
    
    # 读取现有偏好
    existing = store.get((user_id, "preferences"), "data")
    prefs = existing.value if existing else {}
    
    # 更新偏好
    prefs[preference_key] = preference_value
    
    # 保存
    store.put((user_id, "preferences"), "data", prefs)
    
    return f"✓ 已保存偏好: {preference_key} = {preference_value}"

# 定义读取偏好的工具
@tool
def get_user_preference(preference_key: str, runtime: ToolRuntime[UserContext]) -> str:
    """从长期记忆读取用户偏好"""
    user_id = runtime.context.user_id
    store = runtime.store
    
    # 读取偏好
    item = store.get((user_id, "preferences"), "data")
    
    if not item:
        return "未找到任何偏好"
    
    prefs = item.value
    value = prefs.get(preference_key)
    
    if value:
        return f"{preference_key}: {value}"
    else:
        return f"未设置 {preference_key} 偏好"

# 创建Agent
preference_agent = create_agent(
    model=model,
    tools=[save_user_preference, get_user_preference],
    context_schema=UserContext,
    store=user_store,
    checkpointer=InMemorySaver(),  # 同时使用Checkpointer和Store
    system_prompt="""你是偏好管理助手，必须严格遵守以下规则：
1. 当用户要求保存偏好（比如“记住我喜欢Python”“我喜欢简洁的回答”）时，必须调用 save_user_preference 工具，提取清晰的 preference_key 和 preference_value：
   - 例如：“我喜欢Python” → preference_key=编程语言，preference_value=Python；
   - 例如：“喜欢简洁的回答” → preference_key=回答风格，preference_value=简洁。
2. 当用户查询偏好（比如“我喜欢什么编程语言？”“我的回答风格是什么？”）时，必须调用 get_user_preference 工具，传入对应的 preference_key。
3. 工具调用成功后，用自然语言回复用户，禁止仅话术回复而不调用工具。
4. 所有偏好必须通过工具保存到 Store，不能仅靠记忆回复。"""
)

print("✓ Agent创建完成（支持Store和Checkpointer）")

【演示：Agent中使用Store】

✓ Agent创建完成（支持Store和Checkpointer）


### 跨Thread记忆测试

In [9]:
print("【测试：跨Thread记忆】")
print("="*70)

# Thread 1: 用户Alice设置偏好
print("\n【Thread 1：第一次对话】")
response1 = preference_agent.invoke(
    {"messages": [{"role": "user", "content": "帮我记住：我喜欢Python，喜欢简洁的回答"}]},
    config={"configurable": {"thread_id": "alice-thread-1"}},
    context=UserContext(user_id="alice")
)
print(f"用户: 帮我记住：我喜欢Python，喜欢简洁的回答")
print(f"Agent: {response1['messages'][-1].content}")

print("\n" + "-"*70)

# Thread 2: 用户Alice在新对话中查询偏好
print("\n【Thread 2：完全新的对话】")
response2 = preference_agent.invoke(
    {"messages": [{"role": "user", "content": "我喜欢什么编程语言？"}]},
    config={"configurable": {"thread_id": "alice-thread-2"}},  # 不同的thread_id
    context=UserContext(user_id="alice")  # 相同的user_id
)
print(f"用户: 我喜欢什么编程语言？")
print(f"Agent: {response2['messages'][-1].content}")

print("\n" + "="*70)
print("✓ Store让Agent在不同Thread间共享记忆")
print("  - Thread 1保存的偏好")
print("  - Thread 2可以读取")
print("  - 关键：相同的user_id")

【测试：跨Thread记忆】

【Thread 1：第一次对话】
用户: 帮我记住：我喜欢Python，喜欢简洁的回答
Agent: 我已经帮你记住了：你喜欢的编程语言是Python，喜欢的回答风格是简洁。

----------------------------------------------------------------------

【Thread 2：完全新的对话】
用户: 我喜欢什么编程语言？
Agent: 你喜欢的编程语言是Python。还有其他偏好想了解吗？

✓ Store让Agent在不同Thread间共享记忆
  - Thread 1保存的偏好
  - Thread 2可以读取
  - 关键：相同的user_id


## 3. Store的search功能

### 基础search（不带语义）

`store.search(namespace, filter=..., limit=...)`可以搜索namespace内的所有项：

In [10]:
print("【演示：Store的search功能】")
print()

# 创建测试数据
test_store = InMemoryStore()
namespace = ("user-alice", "memories")

test_store.put(namespace, "mem-1", {"text": "Alice喜欢吃披萨", "category": "food"})
test_store.put(namespace, "mem-2", {"text": "Alice是软件工程师", "category": "work"})
test_store.put(namespace, "mem-3", {"text": "Alice喜欢打篮球", "category": "sport"})
test_store.put(namespace, "mem-4", {"text": "Alice喜欢吃寿司", "category": "food"})

print("✓ 创建了4条记忆")
print()

# 基础search：搜索所有记忆
print("【搜索所有记忆】")
items = list(test_store.search(namespace))
print(f"找到 {len(items)} 条记忆")
for item in items:
    print(f"  - {item.value['text']}")
print()

# 带filter的search
print("【搜索category='food'的记忆】")
items = list(test_store.search(namespace, filter={"category": "food"}))
print(f"找到 {len(items)} 条记忆")
for item in items:
    print(f"  - {item.value['text']}")
print()

# 限制数量
print("【搜索最多2条记忆】")
items = list(test_store.search(namespace, limit=2))
print(f"找到 {len(items)} 条记忆")
for item in items:
    print(f"  - {item.value['text']}")

print()
print("说明: search()支持filter和limit参数")

【演示：Store的search功能】

✓ 创建了4条记忆

【搜索所有记忆】
找到 4 条记忆
  - Alice喜欢吃披萨
  - Alice是软件工程师
  - Alice喜欢打篮球
  - Alice喜欢吃寿司

【搜索category='food'的记忆】
找到 2 条记忆
  - Alice喜欢吃披萨
  - Alice喜欢吃寿司

【搜索最多2条记忆】
找到 2 条记忆
  - Alice喜欢吃披萨
  - Alice是软件工程师

说明: search()支持filter和limit参数


### 语义搜索（高级功能）

通过配置embedding模型，Store支持**语义搜索**：

```python
from langchain_openai import OpenAIEmbeddings

# 创建带语义搜索的Store
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
store = InMemoryStore(
    index={
        "embed": embeddings.embed_documents,  # embedding函数
        "dims": 1536  # embedding维度
    }
)

# 使用自然语言查询
items = store.search(
    namespace,
    query="用户喜欢吃什么？",  # 语义查询
    limit=3
)
```

**优势**：
- 不需要精确匹配关键词
- 理解查询的语义
- 返回最相关的记忆

**注意**：本章演示中不使用语义搜索（需要API调用），仅说明概念。

## 4. 实战项目：个性化智能助手

### 项目需求

构建一个记住用户偏好的智能助手：
1. 用户可以告诉助手自己的偏好（语言、风格、兴趣等）
2. 助手在任何对话中都能记住和使用这些偏好
3. 支持多个用户，各自的偏好隔离
4. 偏好永久保存，跨Thread共享

### 系统架构

```
用户 → Agent
         ↓
    ┌────┴────┐
    │         │
Checkpointer  Store
    │         │
  对话历史   用户偏好
 (Thread内)  (跨Thread)
```

In [11]:
print("【实战：个性化智能助手】")
print()

# 定义Context
@dataclass
class AssistantContext:
    user_id: str

# 创建Store和Checkpointer
assistant_store = InMemoryStore()
assistant_checkpointer = InMemorySaver()

# 工具1：保存用户信息
@tool
def remember_user_info(info_key: str, info_value: str, runtime: ToolRuntime[AssistantContext]) -> str:
    """记住用户的信息（偏好、兴趣、基本资料等）"""
    user_id = runtime.context.user_id
    store = runtime.store
    
    # 读取现有信息
    existing = store.get((user_id,), "profile")
    profile = existing.value if existing else {}
    
    # 更新
    profile[info_key] = info_value
    
    # 保存
    store.put((user_id,), "profile", profile)
    
    return f"好的，我记住了：{info_key} = {info_value}"

# 工具2：查询用户信息
@tool
def recall_user_info(info_key: str, runtime: ToolRuntime[AssistantContext]) -> str:
    """查询用户的信息"""
    user_id = runtime.context.user_id
    store = runtime.store
    
    item = store.get((user_id,), "profile")
    
    if not item:
        return f"我还不知道你的 {info_key}"
    
    profile = item.value
    value = profile.get(info_key)
    
    if value:
        return f"你的 {info_key} 是: {value}"
    else:
        return f"我还不知道你的 {info_key}"

# 工具3：获取所有用户信息
@tool
def get_all_user_info(runtime: ToolRuntime[AssistantContext]) -> str:
    """获取用户的所有信息"""
    user_id = runtime.context.user_id
    store = runtime.store
    
    item = store.get((user_id,), "profile")
    
    if not item:
        return "我还没有记录你的任何信息"
    
    profile = item.value
    info_str = "\n".join([f"  - {k}: {v}" for k, v in profile.items()])
    return f"我记住的你的信息：\n{info_str}"

# 创建个性化助手
personal_assistant = create_agent(
    model=model,
    tools=[remember_user_info, recall_user_info, get_all_user_info],
    context_schema=AssistantContext,
    store=assistant_store,
    checkpointer=assistant_checkpointer,
    system_prompt="""你是个性化智能助手。
    
你的特点：
1. 能记住用户告诉你的任何信息（使用remember_user_info工具）
2. 能在需要时查询用户信息（使用recall_user_info工具）
3. 这些信息会永久保存，即使在不同对话中也能记住
4. 根据用户的偏好调整你的回答风格

当用户告诉你关于他们的信息时，主动使用remember_user_info保存。
当需要个性化回答时，先查询用户信息。"""
)

print("✓ 个性化智能助手创建完成")

【实战：个性化智能助手】

✓ 个性化智能助手创建完成


### 测试场景：完整的个性化体验

In [12]:
print("【场景测试：个性化助手】")
print("="*70)

user_context = AssistantContext(user_id="user-chen")

# Day 1, Thread 1: 用户初次使用，告诉助手偏好
print("\n【Day 1, Thread 1：初次使用】")
queries_day1 = [
    "你好！我叫陈明，是一名数据科学家",
    "我喜欢简洁直接的回答，不要太啰嗦",
    "我主要用Python和SQL工作"
]

for query in queries_day1:
    print(f"\n用户: {query}")
    response = personal_assistant.invoke(
        {"messages": [{"role": "user", "content": query}]},
        config={"configurable": {"thread_id": "chen-thread-day1"}},
        context=user_context
    )
    print(f"助手: {response['messages'][-1].content}")

print("\n" + "-"*70)

# Day 2, Thread 2: 新的对话，助手应该记住偏好
print("\n【Day 2, Thread 2：新对话】")
print("\n用户: 我是谁？我做什么工作？")
response = personal_assistant.invoke(
    {"messages": [{"role": "user", "content": "我是谁？我做什么工作？"}]},
    config={"configurable": {"thread_id": "chen-thread-day2"}},  # 新Thread
    context=user_context  # 相同user_id
)
print(f"助手: {response['messages'][-1].content}")

print("\n" + "-"*70)

# Day 3, Thread 3: 要求推荐学习资源
print("\n【Day 3, Thread 3：请求个性化建议】")
print("\n用户: 推荐一些学习资源给我")
response = personal_assistant.invoke(
    {"messages": [{"role": "user", "content": "推荐一些学习资源给我"}]},
    config={"configurable": {"thread_id": "chen-thread-day3"}},  # 又一个新Thread
    context=user_context
)
print(f"助手: {response['messages'][-1].content}")

print("\n" + "="*70)
print("【个性化效果】")
print("✓ Day 1保存的信息，Day 2和Day 3都能读取")
print("✓ 助手在不同Thread中都记得用户偏好")
print("✓ 根据用户职业和技能栈提供个性化建议")
print("✓ Store实现了真正的长期记忆")

【场景测试：个性化助手】

【Day 1, Thread 1：初次使用】

用户: 你好！我叫陈明，是一名数据科学家
助手: 你好，陈明！很高兴认识你。作为一名数据科学家，你有什么特别感兴趣的领域或者项目吗？我可以根据你的兴趣为你提供更个性化的帮助。

用户: 我喜欢简洁直接的回答，不要太啰嗦
助手: 明白了，陈明。我会尽量简洁直接地回答你的问题。有什么需要帮忙的吗？

用户: 我主要用Python和SQL工作
助手: 我知道你主要用Python和SQL工作。需要我帮你写代码示例或解决相关问题吗？

----------------------------------------------------------------------

【Day 2, Thread 2：新对话】

用户: 我是谁？我做什么工作？
助手: 我还不知道你的姓名和工作。你可以告诉我一些关于你自己的信息吗？这样我就能记住并在以后帮助你更好地服务。

----------------------------------------------------------------------

【Day 3, Thread 3：请求个性化建议】

用户: 推荐一些学习资源给我
助手: 为了更好地推荐适合你的学习资源，你能告诉我你感兴趣的学习领域或偏好吗？比如编程、语言学习、历史、科学等。还有你喜欢的视频、书籍、在线课程还是其他形式的资源？这样我能给你更个性化的推荐。

【个性化效果】
✓ Day 1保存的信息，Day 2和Day 3都能读取
✓ 助手在不同Thread中都记得用户偏好
✓ 根据用户职业和技能栈提供个性化建议
✓ Store实现了真正的长期记忆


### 多用户隔离测试

In [13]:
print("【测试：多用户隔离】")
print("="*70)

# 用户A
print("\n【用户A：陈明】")
response_a = personal_assistant.invoke(
    {"messages": [{"role": "user", "content": "告诉我关于我的所有信息"}]},
    config={"configurable": {"thread_id": "test-a"}},
    context=AssistantContext(user_id="user-chen")
)
print(f"助手: {response_a['messages'][-1].content}")

print("\n" + "-"*70)

# 用户B（全新用户）
print("\n【用户B：李华（新用户）】")
print("用户: 你好，我叫李华，是前端开发工程师")
response_b1 = personal_assistant.invoke(
    {"messages": [{"role": "user", "content": "你好，我叫李华，是前端开发工程师"}]},
    config={"configurable": {"thread_id": "test-b1"}},
    context=AssistantContext(user_id="user-li")  # 不同user_id
)
print(f"助手: {response_b1['messages'][-1].content}")

print("\n用户: 告诉我关于我的所有信息")
response_b2 = personal_assistant.invoke(
    {"messages": [{"role": "user", "content": "告诉我关于我的所有信息"}]},
    config={"configurable": {"thread_id": "test-b2"}},  # 新Thread
    context=AssistantContext(user_id="user-li")
)
print(f"助手: {response_b2['messages'][-1].content}")

print("\n" + "="*70)
print("【隔离效果】")
print("✓ 不同user_id的数据完全隔离")
print("✓ 陈明和李华的信息互不干扰")
print("✓ Namespace机制确保数据安全")

【测试：多用户隔离】

【用户A：陈明】
助手: 你告诉我的关于你的信息有：
- 名字是陈明
- 喜欢简洁直接的回答风格
- 编程语言偏好是SQL

如果你想补充或修改信息，随时告诉我。

----------------------------------------------------------------------

【用户B：李华（新用户）】
用户: 你好，我叫李华，是前端开发工程师
助手: 你好，李华！作为一名前端开发工程师，有什么我可以帮你的吗？

用户: 告诉我关于我的所有信息
助手: 你告诉我的关于你的信息有：
- 姓名：李华
- 职业：前端开发工程师

如果你想补充或修改任何信息，随时告诉我！

【隔离效果】
✓ 不同user_id的数据完全隔离
✓ 陈明和李华的信息互不干扰
✓ Namespace机制确保数据安全


## 5. 总结与最佳实践

### 核心要点

#### Store的本质

Store是**跨Thread的长期记忆系统**，用于：
- 存储用户档案、偏好
- 共享知识和上下文
- 实现个性化体验

#### Store vs Checkpointer对比

```
Checkpointer（短期记忆）
  - 作用域：Thread内
  - 内容：对话历史
  - 自动管理：无需显式调用
  - 用途：多轮对话

Store（长期记忆）
  - 作用域：跨Thread
  - 内容：用户偏好、档案
  - 显式管理：需要工具调用
  - 用途：个性化、知识共享
```

#### 完整的Agent记忆系统

```python
agent = create_agent(
    model=model,
    tools=[...],
    checkpointer=InMemorySaver(),  # 短期记忆
    store=InMemoryStore(),         # 长期记忆
    context_schema=Context         # 用户上下文
)

agent.invoke(
    {...},
    config={"configurable": {"thread_id": "xxx"}},  # 短期记忆隔离
    context=Context(user_id="yyy")                    # 长期记忆隔离
)
```

---

### 最佳实践

#### 1. Namespace设计原则

```python
# ✅ 好的设计
# 单层：按用户隔离
namespace = (user_id,)

# 两层：用户+类别
namespace = (user_id, "preferences")
namespace = (user_id, "profile")

# 三层：组织+用户+类别
namespace = (org_id, user_id, "settings")

# ❌ 不好的设计
namespace = ("global",)  # 所有用户共享，不安全
namespace = (user_id, thread_id)  # 错误：thread_id不应该在namespace中
```

#### 2. Key命名规范

```python
# ✅ 好的命名
"profile"          # 用户档案
"preferences"      # 用户偏好
"data"             # 通用数据
f"memory-{uuid}"   # 带唯一ID的记忆

# ❌ 不好的命名
"abc"              # 无意义
"user_data_123"    # 混杂了namespace信息
```

#### 3. 工具设计模式

```python
# 标准的Store工具模式
@tool
def save_data(key: str, value: str, runtime: ToolRuntime[Context]) -> str:
    """保存数据"""
    user_id = runtime.context.user_id  # 从runtime获取用户ID
    store = runtime.store              # 从runtime获取Store
    
    # 读取现有数据（如果需要合并）
    existing = store.get((user_id,), "data")
    data = existing.value if existing else {}
    
    # 更新
    data[key] = value
    
    # 保存
    store.put((user_id,), "data", data)
    
    return f"已保存: {key} = {value}"
```

#### 4. Context Schema设计

```python
@dataclass
class Context:
    user_id: str           # 必需：用于namespace
    org_id: str = None     # 可选：多租户场景
    role: str = None       # 可选：权限控制

# 调用时传入
agent.invoke(
    {...},
    context=Context(
        user_id="user-123",
        org_id="acme",
        role="admin"
    )
)
```

#### 5. 数据结构设计

```python
# ✅ 好的结构：扁平、明确
{
    "name": "Alice",
    "language": "Python",
    "style": "简洁"
}

# ❌ 不好的结构：过度嵌套
{
    "user": {
        "basic": {
            "info": {
                "name": "Alice"
            }
        }
    }
}
```

#### 6. 何时使用Store？

**适合使用Store**：
- ✅ 用户偏好和设置
- ✅ 用户档案和背景
- ✅ 跨对话的知识
- ✅ 需要搜索的记忆

**不适合使用Store**：
- ❌ 对话历史（用Checkpointer）
- ❌ 临时计算结果（用AgentState）
- ❌ 大文件（用专门的文件存储）

#### 7. 开发vs生产环境

```python
import os

if os.getenv("ENV") == "production":
    # 生产环境：使用数据库
    from langgraph.store.postgres import PostgresStore
    store = PostgresStore(connection_string=DB_URI)
else:
    # 开发环境：使用内存
    from langgraph.store.memory import InMemoryStore
    store = InMemoryStore()
```

---

### 常见问题

#### Q1: Store和Checkpointer必须一起使用吗？

**不是必须的**，但推荐一起使用：
- 只用Checkpointer：只有Thread内记忆
- 只用Store：可以跨Thread，但没有对话历史
- 两者都用：完整的记忆系统（推荐）

#### Q2: 如何清理Store中的旧数据？

```python
# 删除指定Key
store.delete((user_id,), "old_key")

# 或在数据库层面设置TTL
```

#### Q3: Store支持事务吗？

`InMemoryStore`不支持。生产环境的`PostgresStore`等支持数据库事务。

#### Q4: 语义搜索必须使用吗？

**不是必须的**。如果不配置embedding，search只能基于filter和limit，不支持语义查询。

#### Q5: 多个Agent可以共享同一个Store吗？

**可以**。这正是Store的设计目的之一，通过namespace隔离数据。

---

### 进阶话题

#### 1. 语义搜索实现

```python
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

store = InMemoryStore(
    index={
        "embed": embeddings.embed_documents,
        "dims": 1536
    }
)

# 存储时自动embedding
store.put(namespace, key, {"text": "用户喜欢吃披萨"})

# 语义搜索
items = store.search(
    namespace,
    query="用户的饮食偏好是什么？",  # 自然语言
    limit=3
)
```

#### 2. 动态System Prompt

根据Store中的用户偏好动态调整system prompt：

```python
from langchain.agents.middleware import wrap_model_call

@wrap_model_call
def dynamic_prompt(request, handler):
    user_id = request.runtime.context.user_id
    store = request.runtime.store
    
    # 读取用户偏好
    prefs = store.get((user_id,), "preferences")
    
    # 动态调整prompt
    if prefs and prefs.value.get("style") == "简洁":
        request = request.override(
            system_prompt=request.system_prompt + "\n请使用简洁的风格回答。"
        )
    
    return handler(request)

agent = create_agent(
    model=model,
    tools=[...],
    middleware=[dynamic_prompt],
    store=store
)
```

#### 3. 跨Agent共享Store

```python
shared_store = InMemoryStore()

agent_a = create_agent(..., store=shared_store)
agent_b = create_agent(..., store=shared_store)

# 两个Agent可以读写同一个Store
```

---

## 下一步学习

完成Store学习后，建议学习：

1. **第17章：消息管理（Memory Management）**
   - Trim策略：移除旧消息
   - Summarize策略：总结历史
   - 上下文窗口管理

2. **生产实践**
   - PostgresStore的使用
   - Store的监控和优化
   - 数据备份和迁移

---

## 关键收获

✅ **Store实现跨Thread的长期记忆**，补充Checkpointer的短期记忆

✅ **Namespace组织数据**，实现多用户隔离和层级管理

✅ **put/get/search三大操作**，简单而强大

✅ **通过ToolRuntime访问Store**，在工具中读写长期记忆

✅ **支持语义搜索**（可选），实现智能记忆检索

✅ **Checkpointer + Store = 完整记忆系统**，实现真正智能的个性化Agent